<a href="https://colab.research.google.com/github/eentartetekunst/NGS_bioinformatics/blob/main/HW1_Hisat2_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
! pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

In [ ]:
!conda install -y -c bioconda samtools

In [ ]:
! sudo apt install bedtools 

In [ ]:
! conda install -c bioconda regtools 

## import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir reads

mkdir: cannot create directory ‘reads’: File exists


In [ ]:
! mv **/*.fq.gz /content/drive/MyDrive/rna.seq.data reads

mv: cannot stat '**/*.fq.gz': No such file or directory


In [ ]:
! ls reads/rna.seq.data

B14.5.fq.gz  B17.5.fq.gz  B34.fq.gz    C15.5.fq.gz  C20.fq.gz
B15.5.fq.gz  B20.fq.gz	  C14.5.fq.gz  C17.5.fq.gz  C34.fq.gz


## 2) install hisat2

In [ ]:
! sudo apt-get update

In [ ]:
! sudo apt-get -y install hisat2

## 3) download chr19 sequence and its annotation

In [ ]:
! wget http://ftp.ensembl.org/pub/release-107/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.chromosome.19.fa.gz #chr19 sequence
! wget http://ftp.ensembl.org/pub/release-107/gtf/mus_musculus/Mus_musculus.GRCm39.107.gtf.gz # annotation
! gzip -d Mus_musculus.GRCm39.dna.chromosome.19.fa.gz
! gzip -d Mus_musculus.GRCm39.107.gtf.gz

## 4) select chr 19 from the annotation

In [ ]:
! touch chr19-annotation 
! grep -P '^19\t' /content/Mus_musculus.GRCm39.107.gtf > chr19-annotation 

In [ ]:
!ls -lh

total 918M
-rw-r--r-- 1 root root  27M Oct  4 08:14 chr19-annotation
-rw-r--r-- 1 root root  16K Oct  4 08:11 condacolab_install.log
drwx------ 5 root root 4.0K Oct  4 08:13 drive
-rw-r--r-- 1 root root 831M Jun  4 09:40 Mus_musculus.GRCm39.107.gtf
-rw-r--r-- 1 root root  60M Jun  4 08:49 Mus_musculus.GRCm39.dna.chromosome.19.fa
drwxr-xr-x 3 root root 4.0K Oct  4 08:13 reads
drwxr-xr-x 1 root root 4.0K Sep 26 13:45 sample_data


In [ ]:
! head -n2 chr19-annotation 

19	ensembl	gene	56265950	56266077	.	-	.	gene_id "ENSMUSG00000077223"; gene_version "3"; gene_name "Gm22271"; gene_source "ensembl"; gene_biotype "snoRNA";
19	ensembl	transcript	56265950	56266077	.	-	.	gene_id "ENSMUSG00000077223"; gene_version "3"; transcript_id "ENSMUST00000104035"; transcript_version "3"; gene_name "Gm22271"; gene_source "ensembl"; gene_biotype "snoRNA"; transcript_name "Gm22271-201"; transcript_source "ensembl"; transcript_biotype "snoRNA"; tag "basic"; transcript_support_level "NA (assigned to previous version 1)";


## 5) build index for chr 19

## map all reads on chr19 sequence (no trimming, known splice sites)
use --known-splicesite-infile:
With this mode, you can provide a list of known splice sites, which HISAT makes use of to align reads with small anchors.
You can create such a list using python extract_splice_sites.py genes.gtf > splicesites.txt, where extract_splice_sites.py is included in the HISAT package, genes.gtf is a gene annotation file, and splicesites.txt is a list of splice sites with which you provide HISAT in this mode

## install extract_splice_sites.py and extract_exons.py

In [ ]:
! wget https://raw.githubusercontent.com/dekoning-lab/ezmap/master/tools/HISAT2/hisat2-2.0.5-Linux/extract_splice_sites.py
! chmod a+x extract_splice_sites.py
! ./extract_splice_sites.py

In [ ]:
! wget https://raw.githubusercontent.com/dekoning-lab/ezmap/master/tools/HISAT2/hisat2-2.0.5-Linux/extract_exons.py
! chmod a+x extract_exons.py
! ./extract_exons.py

## extract splice sites and exon's locations  from the annotation

In [ ]:
! ./extract_splice_sites.py chr19-annotation > chr19.ss # extract sites from the annotation
! ! ./extract_exons.py chr19-annotation > chr19.exons 

In [ ]:
! head -n2 chr19.ss 

19	3103277	3109879	-
19	3109990	3119366	-


In [ ]:
! head -n2 chr19.exons

19	3103070	3103277	-
19	3109879	3109990	-


## map reads on chr19 with no trimming, with known splice sites

---



## map b14.5 on chr19

In [ ]:
! hisat2-build --ss chr19.ss --exon chr19.exons Mus_musculus.GRCm39.dna.chromosome.19.fa base-index



1. --summary-file

3. --no-softclip Disallow soft-clipping.

4. --no-unal Suppress SAM records for reads that failed to align.


5. -x The basename of the index for the reference genome. The basename is the name of any of the index files up to but not including the final

7.  -U Comma-separated list of files containing unpaired reads to be aligned, e.g. lane1.fq,lane2.fq,lane3.fq,lane4.fq. Reads may be a mix of different lengths. If - is specified, hisat2 gets the reads from the “standard in” or “stdin” filehandle.


8.  $fq/$i\.fq.gz # в папке fq с ридами взять файл i в формате fq.gz?
9. samtools view -Sb  (-Output in the BAM format)
11. $i\.bam записываем в файл {read_filename}.bam



In [83]:
i='B14.5'
index='base-index'
fq='/content/reads/rna.seq.data'

In [84]:
# redirect stdout of hisat2 (sam alignment file) to samtools view so that we only save compressed file 
! hisat2 --summary-file $i\.log --no-softclip --no-unal -x $index -U $fq/$i\.fq.gz | samtools view -Sb - > $i\.bam

599672 reads; of these:
  599672 (100.00%) were unpaired; of these:
    47202 (7.87%) aligned 0 times
    540491 (90.13%) aligned exactly 1 time
    11979 (2.00%) aligned >1 times
92.13% overall alignment rate


## 6) Прокартируйте все fq файлы (начните с одного) на 19ую хромосому при помощи hisat2 не допуская обрезания ридов и сообщив hisat2 координаты сайтов сплайсинга 

In [ ]:
%%shell 
reads_list='C14.5 B34 B14.5 C15.5 C17.5 B17.5 C34 B20 B15.5 C20' 
index='base-index'
fq='/content/reads/rna.seq.data'

for i in ${reads_list}
do hisat2 --summary-file ${i}.log --no-softclip --no-unal -x ${index} -U ${fq}/${i}.fq.gz | samtools view -Sb - > ${i}.bam 
done

## 7) Выберите случайно один образец

## 8)Сколько ридов картируется в регион 19:12485000-12490000 в этом образце? 
599672 reads;

## 9)Сколько из них картируются только в одно место генома?

540491 (90.13%) aligned exactly 1 time # из отчета


In [ ]:
! samtools sort -o B14.5.s.bam B14.5.bam #coordinate sorting
# ! mv B14.5.s.bam B14.5.bam # replace unsorted file with sorted 
! samtools index B14.5.s.bam # Generate BAI-format index for BAM files

In [ ]:
! samtools view B14.5.s.bam 19:12485000-12490000 | cut -f6 | grep N

In [ ]:
! samtools view B14.5.s.bam -d NH:1 -c  # тоже самое - риды, которые картировлаись на одно место в геноме

540491


In [ ]:
! samtools view B14.5.s.bam 19:12485000-12490000 -d NH:1 -c  # for a given region

1


то же самое для образца В20

In [ ]:
! samtools sort -o B20.s.bam B20.bam 
! samtools index B20.s.bam

In [ ]:
! samtools view B20.s.bam -d NH:1 -c  # тоже самое - риды, которые картировлаись на одно место в геноме

538522


## 10) Сколько ридов картровалось без замен? Сколько с 1, 2 и т. д. заменами?

 Convert a BAM file to a CRAM with NM and MD tags stored verbatim rather than calculating on the fly during CRAM decode, so that mixed data sets with MD/NM only on some  records, or NM calculated using different definitions of mismatch, can be decoded without change.
         The second command demonstrates how to decode such a file.  The request to not decode MD here is turning off auto-generation of both MD and NM; it will still emit the MD/NM tags
         on records that had these stored verbatim.

           samtools view -C --output-fmt-option store_md=1 --output-fmt-option store_nm=1 -o aln.cram aln.bam
           samtools view --input-fmt-option decode_md=0 -o aln.new.bam aln.cram

In [ ]:
! samtools view B14.5.s.bam -d NM:1 -c # с 1 заменой (наверное )

84502


In [ ]:
! samtools view B14.5.s.bam -d NM:2 -c # с 2 заменами (наверное )

20990


In [ ]:
%%shell
list_my_i='1 2 3 4 5 6 7 8 9 10 11 12'
for i in ${list_my_i}
do samtools view B14.5.s.bam  -d NM:${i} -c # с i заменами 
done

84502
20990
9620
2759
1451
771
344
125
52
16
0
0


## 11) Сколько ридов картировалось на экзон-экзонные границы? 

https://regtools.readthedocs.io/en/latest/commands/junctions-extract/ 

The junctions extract command can be used to extract exon-exon junctions from an RNAseq BAM file. The output is a BED file in the BED12 format

In [ ]:
#reindex
#  rm B14.5.bam.bai
! samtools sort -o B14.5.s.bam B14.5.bam #coordinate sorting
# ! mv B14.5.s.bam B14.5.bam # replace unsorted file with sorted 
! samtools index B14.5.s.bam 

In [ ]:
# /content/B14.5.bam.bai path to coordinate-sorted file
! regtools junctions extract -o B14.5.bed -s XS B14.5.s.bam 


Program:	regtools
Version:	0.5.2
Minimum junction anchor length: 8
Minimum intron length: 70
Maximum intron length: 500000
Alignment: B14.5.s.bam
Output file: B14.5.bed



In [ ]:
! head B14.5.bed

19	3133277	3137371	JUNC00000001	2	+	3133277	3137371	255,0,0	2	64,41	0,4053
19	3311538	3312066	JUNC00000002	8	-	3311538	3312066	255,0,0	2	97,92	0,436
19	3312112	3315585	JUNC00000003	1	-	3312112	3315585	255,0,0	2	32,69	0,3404
19	3315576	3316407	JUNC00000004	6	-	3315576	3316407	255,0,0	2	89,72	0,759
19	3316371	3317307	JUNC00000005	3	-	3316371	3317307	255,0,0	2	88,66	0,870
19	3317264	3318347	JUNC00000006	2	-	3317264	3318347	255,0,0	2	72,50	0,1033
19	3318323	3318700	JUNC00000007	4	-	3318323	3318700	255,0,0	2	93,41	0,336
19	3318804	3321620	JUNC00000008	3	-	3318804	3321620	255,0,0	2	38,95	0,2721
19	3321625	3322913	JUNC00000009	1	-	3321625	3322913	255,0,0	2	75,26	0,1262
19	3322948	3324441	JUNC00000010	13	-	3322948	3324441	255,0,0	2	87,86	0,1407


regtools junctions annotate [options] junctions.bed ref.fa annotations.gtf



In [ ]:
! regtools junctions annotate -o B14.5.exonexon.reads B14.5.bed Mus_musculus.GRCm39.dna.chromosome.19.fa chr19-annotation 

In [ ]:
! head B14.5.exonexon.reads
# score - The number of reads supporting the junction. [integer]

chrom	start	end	name	score	strand	splice_site	acceptors_skipped	exons_skipped	donors_skipped	anchor	known_donor	known_acceptor	known_junction	gene_names	gene_ids	transcripts
19	3133341	3137331	JUNC00000001	2	+	GT-AG	0	0	0	N	0	0	0	NA	NA	NA
19	3311635	3311975	JUNC00000002	8	-	GT-AG	0	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292
19	3312144	3315517	JUNC00000003	1	-	GT-AG	0	0	3	A	0	1	0	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292,ENSMUST00000154537
19	3315665	3316336	JUNC00000004	6	-	GT-AG	0	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292,ENSMUST00000143079,ENSMUST00000154537
19	3316459	3317242	JUNC00000005	3	-	GT-AG	0	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292,ENSMUST00000143079,ENSMUST00000154537
19	3317336	3318298	JUNC00000006	2	-	GT-AG	1	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292,ENSMUST00000154537
19	3318416	3318660	JUNC00000007	4	-	GT

In [ ]:
! cat B14.5.exonexon.reads | awk '{SUM+=$score}END{print SUM}'
# count sum of the coulumn values, $score represent value of column 'score'

109117


109117 reads mapped over exon exon junctions

## 12) Перечислите координаты всех интронов в данном интервале подтверждённых хотя бы одним ридом в формате: chr:from-to coverage

### let’s find all of the non-exonic (i.e., intronic or intergenic) regions of the genome. Note, to do this you need a “genome” file, which tells bedtools the length of each chromosome in your file. Consider why the tool would need this information…

In [ ]:
# $ cat file.gtf | awk '{print $1,$4,$5,"name",$6,$7}' # gtf -> bed basically

In [ ]:
# ! awk -F"\t" '$1 == "chr19" { print $1"\t"$3 }' introns.bed 
# retrieve start and stop coordinates of the introns

In [ ]:
! samtools sort -o B20.s.bam B20.bam #coordinate sorting
# ! mv B14.5.s.bam B14.5.bam # replace unsorted file with sorted 
! samtools index B20.s.bam # Generate BAI-format index for BAM files
! samtools view B20.s.bam  19:12485000-12490000 | cut -f6 | grep N

42M56847N42M50311N17M
42M56845N42M50311N17M
42M56843N42M50311N17M
42M56841N42M50311N17M


In [ ]:
! samtools view B20.s.bam -b 19:12485000-12490000 > targetB20.s.bam # choose target region

In [ ]:
! bedtools bamtobed -i targetB20.s.bam > targetB20.s.bed 

In [ ]:
! head targetB20.s.bed # reads in target region ?

19	12479311	12586570	HISEQ:129:C2LRCACXX:1:2110:5448:97880	0	-
19	12479313	12586570	HISEQ:129:C2LRCACXX:1:2110:5448:97880	0	-
19	12479315	12586570	HISEQ:129:C2LRCACXX:1:2110:5448:97880	0	-
19	12479317	12586570	HISEQ:129:C2LRCACXX:1:2110:5448:97880	0	-
19	12486734	12486835	HISEQ:129:C2LRCACXX:1:2108:20530:65021	60	-
19	12487195	12487296	HISEQ:129:C2LRCACXX:1:1204:10203:87648	60	-
19	12487195	12487296	HISEQ:129:C2LRCACXX:1:2104:14666:2583	60	-


### bedtools complement -i exons.bed -g genome.txt > non-exonic.bed -- to get intron coordinates 

In [ ]:
! head /content/chr19.exons

19	3103070	3103277	-
19	3109879	3109990	-
19	3115710	3118148	-
19	3119366	3119439	-
19	3122687	3122952	-
19	3125713	3125884	-
19	3184644	3184958	+
19	3186983	3187089	-
19	3192759	3193064	+
19	3195867	3196264	-


Если вычести из всех ридов те, что попадают на экзоны, мы получим покрытие интронов:

In [ ]:
! bedtools subtract -a targetB20.s.bed -b chr19.exons 

Координаты интронных ридов:

In [ ]:
! bedtools subtract -a targetB20.s.bed -b chr19.exons  > introns.bed 
! awk -F"\t" '$5 > 0 { print $2"\t"$3 }' introns.bed 
# retrieve introns in the region, covered by at least one read 
# $5 > 0 stands for coverage score

12486734	12486835
12487195	12487296
12487195	12487296
